# 01: Search Fundamentals

## Learning Objectives
- Understand the components of a search problem
- Learn about state spaces and search trees
- Implement a basic search problem
- Visualize search spaces and solutions

## 1. Introduction to Search Problems

Search is fundamental to AI. A search problem consists of:
- **Initial state**: Where we start
- **Actions**: What we can do in each state
- **Transition model**: How actions change states
- **Goal test**: How we know we've succeeded
- **Path cost**: Cost of a sequence of actions

In [ ]:
# Import required modules
import sys
sys.path.append('../src')  # Adjust path as needed

from typing import Any, List, Tuple
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.patches as patches

## 2. The SearchProblem Interface

Let's implement the basic `SearchProblem` class:

In [ ]:
class SearchProblem:
    """Abstract base class for search problems"""
    
    def get_start_state(self) -> Any:
        """Returns the initial state for the search problem"""
        raise NotImplementedError
    
    def is_goal_state(self, state: Any) -> bool:
        """Returns True if the state is a goal state"""
        raise NotImplementedError
    
    def get_successors(self, state: Any) -> List[Tuple[Any, str, float]]:
        """Returns list of (successor, action, cost) tuples"""
        raise NotImplementedError
    
    def get_cost_of_actions(self, actions: List[str]) -> float:
        """Returns the total cost of a sequence of actions"""
        raise NotImplementedError

## 3. Example: Grid Navigation Problem

Let's create a concrete implementation - navigating a grid from start to goal:

In [ ]:
class GridSearchProblem(SearchProblem):
    """Grid navigation with obstacles"""
    
    def __init__(self, grid: List[List], start: Tuple[int, int], goal: Tuple[int, int]):
        """
        Args:
            grid: 2D array where 0=free, 1=obstacle
            start: Starting position (row, col)
            goal: Goal position (row, col)
        """
        self.grid = grid
        self.start = start
        self.goal = goal
        self.rows = len(grid)
        self.cols = len(grid[0]) if grid else 0
    
    def get_start_state(self):
        return self.start
    
    def is_goal_state(self, state):
        return state == self.goal
    
    def get_successors(self, state):
        """Get valid moves from current state"""
        successors = []
        row, col = state
        
        # Define possible moves: up, down, left, right
        moves = [
            ((-1, 0), 'UP', 1.0),
            ((1, 0), 'DOWN', 1.0),
            ((0, -1), 'LEFT', 1.0),
            ((0, 1), 'RIGHT', 1.0)
        ]
        
        for (dr, dc), action, cost in moves:
            new_row, new_col = row + dr, col + dc
            
            # Check if move is valid
            if (0 <= new_row < self.rows and 
                0 <= new_col < self.cols and 
                self.grid[new_row][new_col] != 1):
                
                successors.append(((new_row, new_col), action, cost))
        
        return successors
    
    def get_cost_of_actions(self, actions):
        """Calculate total cost of action sequence"""
        return len(actions)  # Each action costs 1

## 4. Visualizing the Search Space

In [ ]:
def visualize_grid(grid, start=None, goal=None, path=None, title="Grid World"):
    """Visualize a grid with optional start, goal, and path"""
    rows, cols = len(grid), len(grid[0])
    
    fig, ax = plt.subplots(figsize=(cols, rows))
    
    # Create color map
    display = np.zeros((rows, cols, 3))
    
    for i in range(rows):
        for j in range(cols):
            if grid[i][j] == 1:  # Obstacle
                display[i, j] = [0, 0, 0]  # Black
            else:  # Free space
                display[i, j] = [1, 1, 1]  # White
    
    # Mark path if provided
    if path:
        for pos in path:
            if pos != start and pos != goal:
                display[pos[0], pos[1]] = [0.7, 0.7, 1]  # Light blue
    
    # Mark start and goal
    if start:
        display[start[0], start[1]] = [0, 1, 0]  # Green
    if goal:
        display[goal[0], goal[1]] = [1, 0, 0]  # Red
    
    ax.imshow(display)
    
    # Add grid lines
    for i in range(rows + 1):
        ax.axhline(i - 0.5, color='gray', linewidth=0.5)
    for j in range(cols + 1):
        ax.axvline(j - 0.5, color='gray', linewidth=0.5)
    
    # Add coordinates
    for i in range(rows):
        for j in range(cols):
            if grid[i][j] != 1:
                ax.text(j, i, f'({i},{j})', ha='center', va='center', fontsize=8, alpha=0.5)
    
    ax.set_title(title)
    ax.set_xticks([])
    ax.set_yticks([])
    
    # Add legend
    legend_elements = [
        patches.Patch(color='green', label='Start'),
        patches.Patch(color='red', label='Goal'),
        patches.Patch(color='black', label='Obstacle'),
        patches.Patch(color='lightblue', label='Path')
    ]
    ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1, 1))
    
    plt.tight_layout()
    plt.show()

## 5. Create and Visualize a Sample Problem

In [ ]:
# Create a sample grid world
# 0 = free space, 1 = obstacle
sample_grid = [
    [0, 0, 0, 0, 0],
    [0, 1, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

start_pos = (0, 0)
goal_pos = (4, 4)

# Create the problem
problem = GridSearchProblem(sample_grid, start_pos, goal_pos)

# Visualize
visualize_grid(sample_grid, start_pos, goal_pos, title="Sample Grid Problem")

## 6. Exploring the State Space

Let's explore how the search problem generates successors:

In [ ]:
# Get successors from the start state
start_state = problem.get_start_state()
print(f"Start state: {start_state}")
print(f"Is goal? {problem.is_goal_state(start_state)}")
print("\nSuccessors from start:")

for successor, action, cost in problem.get_successors(start_state):
    print(f"  {action}: move to {successor} (cost={cost})")

In [ ]:
# Explore multiple steps
def explore_states(problem, start, depth=2):
    """Explore states up to given depth"""
    visited = set()
    frontier = [(start, 0, [])]  # (state, depth, path)
    state_tree = {}
    
    while frontier:
        state, d, path = frontier.pop(0)
        
        if state in visited or d > depth:
            continue
        
        visited.add(state)
        state_tree[state] = []
        
        if d < depth:
            for successor, action, cost in problem.get_successors(state):
                state_tree[state].append((successor, action))
                frontier.append((successor, d + 1, path + [action]))
    
    return state_tree, visited

# Explore the state space
state_tree, visited_states = explore_states(problem, start_pos, depth=2)

print(f"States reachable within 2 steps: {len(visited_states)}")
print("\nState tree (depth 2):")
for state, successors in state_tree.items():
    if successors:
        print(f"{state}: {successors}")

## 7. Search Tree vs Search Graph

Understanding the difference between search trees and search graphs is crucial:

In [ ]:
class SearchNode:
    """Node in the search tree"""
    
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0 if parent is None else parent.depth + 1
    
    def get_path(self):
        """Return path from root to this node"""
        path = []
        node = self
        while node:
            path.append(node.state)
            node = node.parent
        return list(reversed(path))
    
    def get_actions(self):
        """Return action sequence from root to this node"""
        actions = []
        node = self
        while node.parent:
            actions.append(node.action)
            node = node.parent
        return list(reversed(actions))

# Demonstrate search tree construction
root = SearchNode(problem.get_start_state())
print(f"Root node: state={root.state}, depth={root.depth}")

# Expand root
children = []
for successor, action, cost in problem.get_successors(root.state):
    child = SearchNode(successor, parent=root, action=action, path_cost=root.path_cost + cost)
    children.append(child)
    print(f"Child: state={child.state}, action={child.action}, depth={child.depth}, path_cost={child.path_cost}")

## 8. Key Concepts Summary

### State Space
- **State**: A configuration of the world
- **State Space**: All possible states
- **Initial State**: Where we start
- **Goal State(s)**: Where we want to be

### Search Tree
- **Nodes**: States in the context of a path
- **Root**: Initial state
- **Children**: Successors via actions
- **Path**: Sequence from root to node

### Important Properties
- **Completeness**: Will find solution if it exists
- **Optimality**: Will find best solution
- **Time Complexity**: How many nodes explored
- **Space Complexity**: How many nodes in memory

## 9. Practice Exercises

### Exercise 1: 8-Puzzle Problem
Implement a search problem for the 8-puzzle (sliding tile puzzle):

In [ ]:
class EightPuzzleProblem(SearchProblem):
    """
    8-puzzle: 3x3 grid with tiles 1-8 and one blank (0)
    Goal: Arrange tiles in order
    """
    
    def __init__(self, initial_state, goal_state=None):
        """
        State representation: tuple of 9 numbers (0-8)
        Example: (1, 2, 3, 4, 5, 6, 7, 8, 0) is the goal
        """
        self.initial = initial_state
        self.goal = goal_state or (1, 2, 3, 4, 5, 6, 7, 8, 0)
    
    def get_start_state(self):
        return self.initial
    
    def is_goal_state(self, state):
        return state == self.goal
    
    def get_successors(self, state):
        successors = []
        blank_pos = state.index(0)
        row, col = blank_pos // 3, blank_pos % 3
        moves = [(-1, 0, 'UP'), (1, 0, 'DOWN'), (0, -1, 'LEFT'), (0, 1, 'RIGHT')]
        for dr, dc, action in moves:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                new_pos = new_row * 3 + new_col
                new_state = list(state)
                new_state[blank_pos], new_state[new_pos] = new_state[new_pos], new_state[blank_pos]
                successors.append((tuple(new_state), action, 1))
        return successors

# Test your implementation
puzzle = EightPuzzleProblem((1, 2, 3, 4, 0, 6, 7, 5, 8))
print("Successors:", puzzle.get_successors(puzzle.get_start_state()))

### Exercise 2: Variable Cost Problem
Modify the GridSearchProblem to support variable costs (e.g., diagonal moves cost √2):

In [ ]:
class VariableCostGridProblem(GridSearchProblem):
    """
    Grid navigation with variable movement costs
    - Orthogonal moves (up/down/left/right): cost = 1.0
    - Diagonal moves: cost = 1.414 (√2)
    """
    
    def get_successors(self, state):
        successors = []
        row, col = state
        
        # Orthogonal moves
        moves = [
            ((-1, 0), 'UP', 1.0),
            ((1, 0), 'DOWN', 1.0),
            ((0, -1), 'LEFT', 1.0),
            ((0, 1), 'RIGHT', 1.0)
        ]
        
        # Diagonal moves
        diagonal_moves = [
            ((-1, -1), 'UP-LEFT', 1.414),
            ((-1, 1), 'UP-RIGHT', 1.414),
            ((1, -1), 'DOWN-LEFT', 1.414),
            ((1, 1), 'DOWN-RIGHT', 1.414)
        ]
        
        all_moves = moves + diagonal_moves
        
        for (dr, dc), action, cost in all_moves:
            new_row, new_col = row + dr, col + dc
            
            if (0 <= new_row < self.rows and 
                0 <= new_col < self.cols and 
                self.grid[new_row][new_col] != 1):
                
                successors.append(((new_row, new_col), action, cost))
        
        return successors

# Test your implementation
vcg_problem = VariableCostGridProblem(sample_grid, (0, 0), (4, 4))
print("Variable cost successors:", vcg_problem.get_successors((2, 2)))

## 10. Key Takeaways

1. **Search problems** are defined by states, actions, and goals
2. **State space** can be much larger than what we can explore
3. **Search trees** represent paths through the state space
4. **Different problems** require different state representations
5. **Cost functions** determine optimal solutions

## Next Steps
In the next notebook, we'll implement actual search algorithms (BFS and DFS) to solve these problems!